In [3]:
%pip install tensorflow

  Using cached tensorflow-2.16.1-cp311-cp311-win_amd64.whl.metadata (3.5 kB)
  Using cached tensorflow_intel-2.16.1-cp311-cp311-win_amd64.whl.metadata (5.0 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.7-py2.py3-none-any.whl.metadata (849 bytes)
  Using cached gast-0.5.4-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached h5py-3.10.0-cp311-cp311-win_amd64.whl.metadata (2.5 kB)
  Using cached libclang-16.0.6-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached ml_dtypes-0.3.2-cp311-cp311-win_amd64.whl.metadata (20 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached protobuf-4.25.3-cp310-abi3-win_amd64.whl.metadata (541 bytes)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached wrapt-1.16.0-cp311-cp311-win_amd64.whl.metadata (6

In [3]:
%pip install numpy
%pip install matplotlib
%pip install scikit-learn
%pip install keras-tuner

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Data loading and Pre- Processing

In [9]:
# Housekeeping
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [10]:
#defining the dataset path and directories
base_path = 'dataset/chest_xray'
dir_train = os.path.join(base_path, 'train')
dir_val = os.path.join(base_path, 'val')
dir_test = os.path.join(base_path, 'test')


In [12]:
# Now, defining our image data generator with preprocessing

#augumentation
train_dataGen = ImageDataGenerator(
    rescale=1./255, rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2,
    zoom_range=0.2, horizontal_flip= True)


# No Augumentation for test
test_dataGen = ImageDataGenerator(rescale=1./255)

#loading imgages from the directories
train_Gen = train_dataGen.flow_from_directory(dir_train, target_size=(150,150), batch_size=20, class_mode='binary')

valid_Gen = test_dataGen.flow_from_directory(dir_val, target_size=(150,150), batch_size=20, class_mode='binary')

test_Gen = test_dataGen.flow_from_directory(dir_test, target_size=(150,150), batch_size=20, class_mode='binary')



Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


# DenseNet

## Model Building

This employs DenseNet121 as the feature extractor, incorporates custom dense layers, and designates the pre-trained model layers as non-trainable to retain the acquired features.

In [14]:
from tensorflow.keras.applications import DenseNet121 as DN
from tensorflow.keras import layers, models

#now we will be loading the pre=trained DenseNet
base_dense_mod = DN(include_top=False, weights='imagenet', input_shape=(150, 150,3))

#customized layers
x = base_dense_mod.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256,activation='relu')(x)
predict = layers.Dense(1, activation= 'sigmoid')(x)

model = models.Model(inputs= base_dense_mod.input, outputs= predict)

# freezing the base model layers
for layer in base_dense_mod.layers:
    layer.trainable = False

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


## Model Compilation

Here, the accuracy metric, binary cross-entropy loss function (fit for binary classification tasks), and Adam optimizer are used to create the model.




In [16]:
"""Defining the optimizer, loss function, and metrics"""

from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics= ['accuracy'])
